In [ ]:
import shutil
import json
import csv
import pandas as pd
import os

In [ ]:
!git clone https://github.com/JaidedAI/EasyOCR


Cloning into 'EasyOCR'...
remote: Enumerating objects: 2138, done.
remote: Total 2138 (delta 0), reused 0 (delta 0), pack-reused 2138
Receiving objects: 100% (2138/2138), 147.89 MiB | 15.05 MiB/s, done.
Resolving deltas: 100% (1317/1317), done.
Checking out files: 100% (225/225), done.


###Приведение датасета к виду: картинки в одной папке, надписи - в csv файле, содержащим 2 столбца: название картинки, надпись, содержащаяся в картинке

Распаковка

In [ ]:
shutil.unpack_archive('/content/drive/MyDrive/Norbit_project/autoriaNumberplateOcrRu-2021-09-01.zip', '/content/dataset/', 'zip' )

1. Получить все названия файлов
2. открывать их в цикле и дергать дескрипшн
3. записывать в csv файл

###Конвертация png в jpg

In [ ]:
from PIL import Image

In [ ]:
dirname = "/content/dataset/autoriaNumberplateOcrRu-2021-09-01/val/img"

In [ ]:
files = os.listdir(dirname)

In [ ]:
print(files[:10])

['Y951CT62.jpg', 'O005BY199.jpg', 'O330PX42.jpg', 'H404CC26.jpg', 'H245AH154.jpg', 'T800HT93_0.jpg', 'B492CX47.jpg', 'H915PY69.jpg', 'K904KX10_0.jpg', 'C160KA77.jpg']


In [ ]:
for file_ in files:
  file_path = os.path.join(dirname, file_)
  if file_.endswith(".png"):
    img = Image.open(file_path)
    rgb_im = img.convert('RGB')
    os.remove(file_path)
    rgb_im.save(file_path[:-3] + 'jpg')

###Создание labels.csv

In [ ]:
dirname = "/content/dataset/autoriaNumberplateOcrRu-2021-09-01/val/ann"

In [ ]:
files = os.listdir(dirname)

In [ ]:
print(files[:10])

['K301HP196.json', 'B139PP178.json', 'X395AO56.json', 'C539AA142.json', 'O988TY31.json', 'O670OP69.json', 'X829KO799.json', 'K373OC29.json', 'C117BK34.json', 'K394AB134.json']


In [ ]:
cols = ['filename', 'words'] 

In [ ]:
with open('/content/number_dataset.csv', 'a', encoding='utf-8') as f2:
      wr = csv.DictWriter(f2, fieldnames = cols) 
      wr.writeheader() 
       
      f2.close()
row = dict()
for file_ in files:
  file_path = os.path.join(dirname, file_)
  if file_.endswith(".json"):
    with open(file_path, 'r', encoding='utf-8') as f: 
      text_ = json.load(f) 
      f.close()

  
    row['filename'] = str(text_.get('name')) + '.jpg'
    row['words'] = str(text_.get('description'))
    print(row)
    with open('/content/number_dataset.csv', 'a', encoding='utf-8') as f2:
      wr = csv.DictWriter(f2, fieldnames = cols) 
      wr.writerow(row) 
      f2.close()
    row.clear()
  







In [ ]:
%mkdir /content/EasyOCR/trainer/all_data/en_train_filtered/

In [ ]:
%mv /content/labels.csv /content/EasyOCR/trainer/all_data/en_train_filtered/

In [ ]:
%mv /content/labels.csv /content/EasyOCR/trainer/all_data/en_val

Copying all the converted jpg files into right folder

In [ ]:
%cp -R /content/dataset/autoriaNumberplateOcrRu-2021-09-01/train/img/. /content/EasyOCR/trainer/all_data/en_train_filtered/

In [ ]:
%mkdir /content/EasyOCR/trainer/all_data/en_val

In [ ]:
dirname = "/content/EasyOCR/trainer/all_data/en_val"

In [ ]:
files = os.listdir(dirname)

In [ ]:
print(files[:10])

['Y951CT62.jpg', 'O005BY199.jpg', 'O330PX42.jpg', 'H404CC26.jpg', 'H245AH154.jpg', 'T800HT93_0.jpg', 'B492CX47.jpg', 'H915PY69.jpg', 'K904KX10_0.jpg', 'C160KA77.jpg']


In [ ]:
%cp -R /content/dataset/autoriaNumberplateOcrRu-2021-09-01/val/img/. /content/EasyOCR/trainer/all_data/en_val/

In [ ]:
%mv /content/labels.csv /content/EasyOCR/trainer/all_data/en_train_filtered

In [ ]:
%cd EasyOCR/trainer/all_data/en_train_filtered/

/content/EasyOCR/trainer/all_data/en_train_filtered


###Training

Нужно настроить конфиг файл 
/content/EasyOCR/trainer/config_files/en_filtered_config.yaml


In [ ]:
ls

custom_model.md  examples/    README.md         scripts/   trainer/
Dockerfile       LICENSE      releasenotes.md   setup.cfg
easyocr/         MANIFEST.in  requirements.txt  setup.py


In [ ]:
cd /content/EasyOCR/trainer/

/content/EasyOCR/trainer


In [ ]:
import os
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
import pandas as pd

In [ ]:
cudnn.benchmark = True
cudnn.deterministic = False

In [ ]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [ ]:
%mv /content/labels.csv /content/dataset/autoriaNumberplateOcrRu-2021-09-01/train/img/

In [ ]:
%mv /content/labels.csv /content/dataset/autoriaNumberplateOcrRu-2021-09-01/val/img/

Конвертация png to jpb (это пиздец, не мог 2 часа ошибку найти... кстати, на данный момент неизвестно, в этом ли ошибка.... Говорит типа num_samples = 0 сначала подумал, что что-то с путями, потом с версиями в торче, ну короче...)

1. конвертировать в jpg
2. удалить labels.csv
3. создать новые labels.csv

In [ ]:
%rm /content/EasyOCR/trainer/all_data/en_train_filtered/labels.csv

Удалить все файлы из trainer

In [ ]:
%cp /content/EasyOCR/trainer/all_data/en_train_filtered/labels.csv /content/dataset/autoriaNumberplateOcrRu-2021-09-01/

In [ ]:
%pwd

'/content/EasyOCR/trainer'

In [ ]:
dirname = "/content/dataset/autoriaNumberplateOcrRu-2021-09-01/train/img"

In [ ]:
files = os.listdir(dirname)

In [ ]:
print(files[:10])

[]


In [ ]:
%rmdir -p /content/dataset

rmdir: failed to remove '/content/dataset': Directory not empty


In [ ]:
for file_ in files:
  file_path = os.path.join(dirname, file_)
  if file_.endswith(".png"):
    #os.remove(file_path)
    print(file_path)


In [ ]:
opt = get_config("config_files/en_filtered_config.yaml")


In [ ]:
print(opt.select_data)

en_train_filtered


In [ ]:
train(opt, amp=False)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: all_data
opt.select_data: ['en_train_filtered']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    all_data	 dataset: en_train_filtered
all_data/en_train_filtered


Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback

sub-directory:	/en_train_filtered	 num samples: 480
num total samples of en_train_filtered: 480 x 1.0 (total_data_usage_ratio) = 480
num samples of en_train_filtered per batch: 32 x 1.0 (batch_ratio) = 32


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


--------------------------------------------------------------------------------
Total_batch_size: 32 = 32
--------------------------------------------------------------------------------
dataset_root:    all_data/en_val	 dataset: /
all_data/en_val/
sub-directory:	/.	 num samples: 60
--------------------------------------------------------------------------------
No Transformation module specified
model input parameters 64 600 20 1 256 256 25 34 None VGG BiLSTM CTC
Model:
DataParallel(
  (module): Model(
    (FeatureExtraction): VGG_FeatureExtractor(
      (ConvNet): Sequential(
        (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): ReLU(inplace=True)
        (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (6): Conv